## Применим к картинкам модель VGG16

In [2]:
'''Импорты для препроцессинга'''
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input

In [3]:
'''Импорты моделей'''
from keras.applications.vgg16 import VGG16 
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [4]:
'''Импорты базовых библиотек для обработки данных'''
import os
import numpy as np
from numpy import savetxt
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import cv2
import re
import csv
from tqdm import tqdm
import collections

In [5]:
'''Импорты для визуализации'''
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.transform import factor_cmap

In [6]:
'''Функция создает список shots, в котором лежат названия файлов кадров'''
def add_image_names_2_list(path):
    # change the working directory to the path where the images are located
    os.chdir(path)

    # this list holds all the image filename
    shots = []

    # creates a ScandirIterator aliased as files
    with os.scandir(path) as files:
      # loops through each file in the directory
        for file in files:
            if file.name.endswith('.jpg'):
              # adds only the image files to the flowers list
                shots.append(file.name)
    return shots

In [7]:
'''Features - признаки объекта, получение сжатия размерности. В данной функции мы загружаем изображение,
предобрабатываем его с помощью встроенных методов keras, изменяем размерность, потому что нейронная сеть работает
только с изображениями определенных форматов'''
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [8]:
'''Путь к нейронке'''
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'

In [9]:
'''Загружаем модель'''
model = VGG16(weights = weights_path)
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

# Выделяем Features для каждого фильма

In [9]:
'''Путь к кадрам фильма'''
path_2012 = '2012'

In [8]:
def get_image_features(path, model):
    shots = add_image_names_2_list(path)
    shots.sort(key=lambda f: int(re.sub('\D', '', f)))
    data = {}

    for shot in tqdm(shots):
        feat = extract_features(shot, model)
        data[shot] = feat

    filenames = np.array(list(data.keys()))

# get a list of just the features
    feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
    feat = feat.reshape(-1,4096)
    
    return data, feat, filenames

## Получение и сохранение даты

In [11]:
'''Энергозатратный чанк, изменяет размерность каждого фрейма'''
data, feat, filenames = get_image_features(path_2012, model)

100%|██████████| 7764/7764 [48:52<00:00,  2.65it/s]  


In [14]:
def save_files_and_features_to_csv(data, year_folder, year_file):
    pd.DataFrame(np.concatenate(list(data.values())), index=data.keys()).to_csv("Data/{}/Features_{}.csv".format(year_folder, year_file))

In [31]:
save_files_and_features_to_csv(data, 2012, 2012)